In [ ]:
print('get all txs')
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import google_bq_utils as bqu
sys.path.pop()

import time

import numpy as np
import pandas as pd
import os

In [2]:
col_list = [
        'dt','blockchain','name','layer','chain_id'
        , 'num_raw_txs', 'num_success_txs','num_qualified_txs','source'
        ]
query_name = 'dune_all_txs'

In [3]:
chain_config = [
    #blockchain, display_name, count_func, gas_field, transactions_table
    ['bitcoin','Bitcoin','count(*)','fee','transactions'],
    ['near','Near','count(distinct tx_hash)','gas_price','actions'],
    ['aptos','Aptos','count(*)','gas_used','transactions'],
]

In [4]:
trailing_days = 180

In [5]:
days_param = d.generate_query_parameter(input=trailing_days,field_name='trailing_days',dtype='number')

In [ ]:
# Run Dune
print('     dune runs')
fees_df = d.get_dune_data(query_id = 4229341, #https://dune.com/queries/4229341
    name = query_name,
    path = "outputs",
    performance="medium",
    params = [days_param],
    num_hours_to_rerun=4
)

In [ ]:
fees_df.sample(5)

In [8]:
unique_blockchains = fees_df['blockchain'].unique().tolist()

In [ ]:
chain_dfs = []
for row in chain_config:
    blockchain = row[0]
    print(blockchain)
    if blockchain in unique_blockchains:
        continue
    else:
        #blockchain, display_name, count_func, gas_field, transactions_table
        blockchain_param = d.generate_query_parameter(input=blockchain,field_name='blockchain',dtype='text')
        display_name_param = d.generate_query_parameter(input=row[1],field_name='display_name',dtype='text')
        count_func_param = d.generate_query_parameter(input=row[2],field_name='count_func',dtype='text')
        gas_field_param = d.generate_query_parameter(input=row[3],field_name='gas_field',dtype='text')
        transactions_table_param = d.generate_query_parameter(input=row[4],field_name='transactions_table',dtype='text')

        chain_df = d.get_dune_data(query_id = 4230061, #https://dune.com/queries/4230061
            name = query_name + '_by_chain',
            path = "outputs",
            performance="medium",
            params = [days_param,blockchain_param,display_name_param,count_func_param,gas_field_param,transactions_table_param],
            num_hours_to_rerun=4
        )
        print(chain_df.sample(3))
        chain_dfs.append(chain_df)

chain_df_agg = pd.concat(chain_dfs)

In [ ]:
print(fees_df.columns)
print(chain_df_agg.columns)

In [ ]:
dune_df = pd.concat([fees_df,chain_df_agg])

In [13]:
if not dune_df.empty:
    dune_df['source'] = 'dune'
    dune_df['dt'] = pd.to_datetime(dune_df['dt']).dt.tz_localize(None)
    dune_df['chain_id'] = dune_df['chain_id'].astype(str)
    dune_df['chain_id'] = dune_df['chain_id'].astype(str).str.replace(r'\.0$', '', regex=True)

In [ ]:
# Verify that all elements are strings
assert dune_df['chain_id'].apply(type).eq(str).all(), "Not all elements are strings"
print(dune_df['chain_id'].dtype)

In [ ]:
# dune_df.sample(5)
print(dune_df.dtypes)

In [ ]:
print(f"dune_df shape: {dune_df.shape}")

In [ ]:
#BQ Upload
bqu.append_and_upsert_df_to_bq_table(dune_df, query_name, unique_keys = ['blockchain','dt'])